In [27]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Softmax

import numpy as np

#array de colunas

In [28]:
weeks1 = pd.read_csv('../data_raw/dados/2016-1/tabela processada nota media mediana iteracao.csv')
weeks2 = pd.read_csv('../data_raw/dados/2016-2/tabela de iteracoes com nota add.csv')
weeks3 = pd.read_csv('../data_raw/dados/2017-1/tabela final.csv')
weeks4 = pd.read_csv('../data_raw/dados/2017-2/tabela com media mediana e interacao e nota.csv')

weeks =  pd.concat([weeks1, weeks2, weeks3, weeks4])

weeks_num = 17

features = []
for i in range(weeks_num):
  features.append(f'Week{i}')  
  features.append(f'MedianaW{i}')
  features.append(f'MediaW{i}')
  
weeks.fillna(0, inplace=True)

In [29]:
#Deleting unused columns
#Converting not number values to numerical values: 1 = disapproved; 0 = approved
weeks.drop(['Unnamed: 0.1','Notas', 'Unnamed: 0', 'Nome'], inplace=True, axis=1)
weeks['Aprovado'] = pd.Categorical(weeks['Aprovado'])
weeks['Aprovado'] = weeks.Aprovado.cat.codes
weeks

,Week0,MedianaW0,MediaW0,Week1,MedianaW1,MediaW1,Week2,MedianaW2,MediaW2,Week3,...,Week15,MedianaW15,MediaW15,Week16,MedianaW16,MediaW16,Week17,MedianaW17,MediaW17,Aprovado
0,15,15.0,15.000000,5,5.0,5.000000,0,0.0,0.0,0,...,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1
1,21,10.5,10.500000,16,5.0,5.333333,3,1.5,1.5,1,...,10,3.0,3.333333,5.0,2.5,2.5,3.0,3.0,3.0,1
2,3,3.0,3.000000,20,9.0,6.666667,1,1.0,1.0,0,...,0,0.0,0.000000,2.0,2.0,2.0,0.0,0.0,0.0,1
3,350,9.0,116.666667,1568,297.0,313.600000,1054,284.0,263.5,13,...,175,87.5,87.500000,1.0,1.0,1.0,1.0,1.0,1.0,0
4,6,6.0,6.000000,11,2.5,2.750000,5,2.5,2.5,1,...,4,1.0,1.333333,4.0,2.0,2.0,3.0,3.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,0,0.0,0.000000,0,0.0,0.000000,2,2.0,2.0,0,...,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1
28,0,0.0,0.000000,0,0.0,0.000000,0,0.0,0.0,0,...,34,17.0,17.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
29,0,0.0,0.000000,0,0.0,0.000000,0,0.0,0.0,0,...,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1
30,0,0.0,0.000000,0,0.0,0.000000,0,0.0,0.0,0,...,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1


In [30]:
train, validation, test = np.split(weeks.sample(frac=1), [int(.7*len(weeks)), int(.85*len(weeks))])

X_train = train[[train.columns[i] for i in range(train.shape[1]-1) ]]
y_train = train[train.columns[-1]]
X_test = test[[test.columns[i] for i in range(train.shape[1]-1) ]]
y_test = test[test.columns[-1]]
X_val = validation[[validation.columns[i] for i in 
                    range(validation.shape[1]-1) ]]
y_val = validation[validation.columns[-1]]
X_train = X_train[features]
X_test = X_test[features]
X_val = X_val[features]
X_train
  

,Week0,MedianaW0,MediaW0,Week1,MedianaW1,MediaW1,Week2,MedianaW2,MediaW2,Week3,...,MediaW13,Week14,MedianaW14,MediaW14,Week15,MedianaW15,MediaW15,Week16,MedianaW16,MediaW16
17,0,0.0,0.0,2,2.0,2.000000,0,0.0,0.000000,0,...,0.000000,0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.000000
4,3,1.5,1.5,50,9.0,12.500000,45,6.5,7.500000,57,...,6.000000,13,13.0,13.0,20,8.0,6.666667,26.0,3.0,6.500000
1,18,18.0,18.0,53,11.0,13.250000,87,4.5,14.500000,72,...,6.000000,6,3.0,3.0,15,7.5,7.500000,7.0,2.0,2.333333
16,2,1.0,1.0,2,1.0,1.000000,1,1.0,1.000000,0,...,0.000000,0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.000000
3,29,29.0,29.0,38,4.0,9.500000,31,7.5,7.750000,30,...,31.500000,19,19.0,19.0,6,2.0,2.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,8,0.0,0.0,2,0.0,0.000000,18,0.0,0.000000,7,...,0.000000,3,0.0,0.0,6,0.0,0.000000,5.0,0.0,0.000000
6,80,5.0,20.0,62,8.0,8.857143,66,6.0,9.428571,59,...,5.333333,30,8.0,10.0,409,204.5,204.500000,0.0,0.0,0.000000
21,13,13.0,13.0,11,11.0,11.000000,0,0.0,0.000000,1,...,0.000000,0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.000000
35,0,0.0,0.0,0,0.0,0.000000,0,0.0,0.000000,0,...,1.000000,0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.000000


In [31]:
model = Sequential()
model.add(LSTM(10, activation='relu', input_shape=(weeks_num*3, 1)))
model.add(Dense(1))
model.add(Softmax(1))
model.compile(optimizer='adam', loss='mse')

In [32]:
model.fit(X_train, y_train, epochs=200, verbose=2)

Epoch 1/200
3/3 - 3s - loss: 0.3176 - 3s/epoch - 975ms/step
Epoch 2/200
3/3 - 0s - loss: 0.3176 - 81ms/epoch - 27ms/step
Epoch 3/200
3/3 - 0s - loss: 0.3176 - 96ms/epoch - 32ms/step
Epoch 4/200
3/3 - 0s - loss: 0.3176 - 187ms/epoch - 62ms/step
Epoch 5/200
3/3 - 0s - loss: 0.3176 - 104ms/epoch - 35ms/step
Epoch 6/200
3/3 - 0s - loss: 0.3176 - 94ms/epoch - 31ms/step
Epoch 7/200
3/3 - 0s - loss: 0.3176 - 89ms/epoch - 30ms/step
Epoch 8/200
3/3 - 0s - loss: 0.3176 - 380ms/epoch - 127ms/step
Epoch 9/200
3/3 - 0s - loss: 0.3176 - 132ms/epoch - 44ms/step
Epoch 10/200
3/3 - 0s - loss: 0.3176 - 90ms/epoch - 30ms/step
Epoch 11/200
3/3 - 0s - loss: 0.3176 - 378ms/epoch - 126ms/step
Epoch 12/200
3/3 - 0s - loss: 0.3176 - 229ms/epoch - 76ms/step
Epoch 13/200
3/3 - 0s - loss: 0.3176 - 297ms/epoch - 99ms/step
Epoch 14/200
3/3 - 0s - loss: 0.3176 - 168ms/epoch - 56ms/step
Epoch 15/200
3/3 - 0s - loss: 0.3176 - 123ms/epoch - 41ms/step
Epoch 16/200
3/3 - 0s - loss: 0.3176 - 159ms/epoch - 53ms/step
Epoch 

In [33]:
score = model.evaluate(X_test, y_test, verbose =2)
print(1-score)

1/1 - 1s - loss: 0.2632 - 632ms/epoch - 632ms/step
0.7368420958518982


In [34]:
score_val = model.evaluate(X_val, y_val)
print(1-score_val)

1/1 [==============================] - 0s 95ms/step - loss: 0.1667
0.8333333283662796
